In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [2]:
def get_url(position, location):
    temp = 'https://www.careerjunction.co.za/jobs/results?keywords={}&location={}'
    url = temp.format(position, location)
    return url

In [3]:
url = get_url('data+scientist', '10860')
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
cards = soup.find_all('div', 'module job-result')
card = cards[0]
atag = card.h2.a
job_title = atag.text
job_title_url = 'https://www.careerjunction.co.za' + atag.get('href')
company = card.h3.a.text
location = card.find('li', 'location').text.strip()
job_summary = card.find('ul', 'job-overview').text.strip()
post_date = card.find('li', 'updated-time').text.strip()
today = datetime.today().strftime('%Y-%m-%d')

In [4]:
def get_record(card):
    atag = card.h2.a
    job_title = atag.text
    job_title_url = 'https://www.careerjunction.co.za' + atag.get('href')
    company = card.h3.a.text
    location = card.find('li', 'location').text.strip()
    job_summary = card.find('ul', 'job-overview').text.strip()
    post_date = card.find('li', 'updated-time').text.strip()
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('li', 'salary').text.strip()
    except AttributeError:
        job_salary = 'R Undisclosed'

    record = (job_title, company, location, job_salary, post_date, today, job_summary, job_title_url)

    return record

In [5]:
records = []
for card in cards:
    record = get_record(card)
    records.append(record)

In [6]:
def main(position, location):
    records = []
    url = get_url(position, location)
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'module job-result')
    
    for card in cards:
        record = get_record(card)
        records.append(record)
             
    with open('career-junction.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)       
        writer.writerow(['JobTitle', 'Company', 'Location', 'Salary', 'PostDate', 'ExtractDate','Summary','JobUrl'])
        writer.writerows(records)
        
main('data+scientist', '10860')

